# Placebo and sensitivity analysis (trustworthiness guardrails)

This notebook focuses on **guardrails** that prevent overconfidence:

1) **Placebo-in-time**: fake intervention dates in the pre-period.
2) **Leave-one-covariate-out**: check robustness to dropping a control.
3) **Drop last K days of pre**: check stability to pre-window choice.
4) **Window sensitivity**: evaluate impact across multiple analysis windows.

We also save plots that can be reused for an article.


In [ ]:
import sys
from pathlib import Path

# Ensure `src/` is importable when running from repo root
repo_root = Path.cwd()
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tecore.causal import DataSpec, ImpactConfig, ImpactMethod, run_impact
from tecore.causal.simulate_ts import SyntheticTSConfig, generate_synthetic_time_series
from tecore.causal.sensitivity import leave_one_covariate_out_test, drop_last_k_days_of_pre_test


## 1) Generate a synthetic dataset with a known effect

In [ ]:
cfg = SyntheticTSConfig(
    n_days=240,
    start_date="2025-01-01",
    intervention_day=150,
    level_shift=10.0,
    slope_change=0.0,
    temp_effect_amp=5.0,
    temp_effect_decay=0.10,
    confounding=False,
    random_state=20,
)

df, meta = generate_synthetic_time_series(cfg)
intervention_date = meta["intervention_date"]
print("Intervention:", intervention_date)
print("True cumulative effect (synthetic):", meta.get("true_cum_effect"))
df.head()

## 2) Run causal impact (with placebo)

In [ ]:
spec = DataSpec(
    date_col="date",
    y_col="y",
    x_cols=["sessions", "active_users", "marketing_spend", "external_index"],
    freq="D",
    add_time_trend=True,
    add_day_of_week=True,
)

cfg_run = ImpactConfig(
    intervention_date=intervention_date,
    method=ImpactMethod.CAUSAL_IMPACT_LIKE,
    ridge_alpha=1.0,
    bootstrap_iters=300,
    block_size=7,
    alpha=0.05,
    run_placebo=True,
    n_placebos=40,
    pre_period_min_points=60,
)

res = run_impact(df, spec, cfg_run)
res.summary()

### Plot: observed vs counterfactual

In [ ]:
eff = res.effect_series.copy()
eff["date"] = pd.to_datetime(eff["date"])

plt.figure()
plt.plot(eff["date"], eff["y"], label="observed")
plt.plot(eff["date"], eff["y_hat"], label="counterfactual")
plt.axvline(pd.Timestamp(intervention_date), linestyle="--")
plt.title("Observed vs counterfactual")
plt.legend()
plt.tight_layout()
plt.show()

### Plot: placebo distribution

In [ ]:
if res.placebo_results is None:
    print("Placebo not available.")
else:
    plt.figure()
    plt.hist(res.placebo_results["cum_effect"].values, bins=18)
    plt.axvline(res.cum_effect, linestyle="--")
    plt.title("Placebo-in-time distribution (cumulative effect)")
    plt.tight_layout()
    plt.show()

    print("Estimated cumulative effect:", res.cum_effect)
    print("Placebo p-value:", res.p_value)

## 3) Sensitivity: leave-one-covariate-out

In [ ]:
loo = leave_one_covariate_out_test(df, spec, cfg_run)
loo

In [ ]:
# Plot as bars for quick visual comparison
plt.figure()
plt.bar(loo["scenario"], loo["cum_effect"])
plt.title("Leave-one-out sensitivity: cumulative effect")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

## 4) Sensitivity: drop last K days of pre

If dropping a small portion of the pre-period changes the estimate dramatically, treat results as fragile.


In [ ]:
stab = drop_last_k_days_of_pre_test(df, spec, cfg_run, k_list=[7, 14, 21, 28])
stab

In [ ]:
plt.figure()
plt.plot(stab["scenario"], stab["cum_effect"], marker="o")
plt.title("Stability test: drop last K days of pre")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

## 5) Window sensitivity

We vary the analysis window (how much pre-history is included) while keeping the intervention date fixed.

This helps detect whether conclusions depend on an arbitrary window choice.


In [ ]:
def run_with_pre_window_days(df_in, intervention_date, pre_days, post_days=60):
    d = df_in.copy()
    d["date"] = pd.to_datetime(d["date"])
    t0 = pd.Timestamp(intervention_date)
    start = t0 - pd.Timedelta(days=pre_days)
    end = t0 + pd.Timedelta(days=post_days)
    d = d[(d["date"] >= start) & (d["date"] <= end)].copy()
    d["date"] = d["date"].dt.strftime("%Y-%m-%d")

    cfg_local = ImpactConfig(
        intervention_date=intervention_date,
        method=ImpactMethod.CAUSAL_IMPACT_LIKE,
        ridge_alpha=1.0,
        bootstrap_iters=200,
        block_size=7,
        alpha=0.05,
        run_placebo=True,
        n_placebos=25,
        pre_period_min_points=max(20, int(pre_days * 0.6)),
    )
    return run_impact(d, spec, cfg_local)

pre_windows = [30, 45, 60, 90, 120]
rows = []
for w in pre_windows:
    r = run_with_pre_window_days(df, intervention_date, pre_days=w, post_days=60)
    rows.append({
        "pre_days": w,
        "cum_effect": r.cum_effect,
        "cum_ci_low": r.cum_ci[0],
        "cum_ci_high": r.cum_ci[1],
        "p_value": r.p_value,
        "warnings": "; ".join(r.warnings) if r.warnings else ""
    })

win = pd.DataFrame(rows)
win

In [ ]:
plt.figure()
plt.errorbar(win["pre_days"], win["cum_effect"], 
             yerr=[win["cum_effect"] - win["cum_ci_low"], win["cum_ci_high"] - win["cum_effect"]],
             fmt="o")
plt.axhline(0.0, linestyle="--")
plt.title("Window sensitivity: cumulative effect vs pre-window length")
plt.xlabel("Pre window (days)")
plt.ylabel("Cumulative effect")
plt.tight_layout()
plt.show()

## 6) Save outputs and figures

This block saves CSV tables and key plots under `out/notebook_13_placebo_sensitivity/`.
You can re-use these images for an article.


In [ ]:
out_dir = repo_root / "out" / "notebook_13_placebo_sensitivity"
out_dir.mkdir(parents=True, exist_ok=True)

# Save tables
eff_to_save = res.effect_series.copy()
eff_to_save["date"] = pd.to_datetime(eff_to_save["date"]).dt.strftime("%Y-%m-%d")
eff_to_save.to_csv(out_dir / "effect_series.csv", index=False)

if res.placebo_results is not None:
    pl = res.placebo_results.copy()
    pl["placebo_date"] = pd.to_datetime(pl["placebo_date"]).dt.strftime("%Y-%m-%d")
    pl.to_csv(out_dir / "placebo_results.csv", index=False)

loo.to_csv(out_dir / "leave_one_out.csv", index=False)
stab.to_csv(out_dir / "drop_last_k_pre.csv", index=False)
win.to_csv(out_dir / "window_sensitivity.csv", index=False)

# Save plots by re-plotting with savefig
def save_fig(path):
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    plt.close()

# Observed vs counterfactual
plt.figure()
plt.plot(eff["date"], eff["y"], label="observed")
plt.plot(eff["date"], eff["y_hat"], label="counterfactual")
plt.axvline(pd.Timestamp(intervention_date), linestyle="--")
plt.title("Observed vs counterfactual")
plt.legend()
save_fig(out_dir / "observed_vs_counterfactual.png")

# Placebo histogram
if res.placebo_results is not None:
    plt.figure()
    plt.hist(res.placebo_results["cum_effect"].values, bins=18)
    plt.axvline(res.cum_effect, linestyle="--")
    plt.title("Placebo-in-time distribution (cumulative effect)")
    save_fig(out_dir / "placebo_hist.png")

# Leave-one-out bars
plt.figure()
plt.bar(loo["scenario"], loo["cum_effect"])
plt.title("Leave-one-out sensitivity: cumulative effect")
plt.xticks(rotation=45, ha="right")
save_fig(out_dir / "leave_one_out.png")

# Drop last K pre
plt.figure()
plt.plot(stab["scenario"], stab["cum_effect"], marker="o")
plt.title("Stability test: drop last K days of pre")
plt.xticks(rotation=45, ha="right")
save_fig(out_dir / "drop_last_k_pre.png")

# Window sensitivity errorbars
plt.figure()
plt.errorbar(win["pre_days"], win["cum_effect"], 
             yerr=[win["cum_effect"] - win["cum_ci_low"], win["cum_ci_high"] - win["cum_effect"]],
             fmt="o")
plt.axhline(0.0, linestyle="--")
plt.title("Window sensitivity: cumulative effect vs pre-window length")
plt.xlabel("Pre window (days)")
plt.ylabel("Cumulative effect")
save_fig(out_dir / "window_sensitivity.png")

print("Wrote:", out_dir)

## Interpretation notes (for an article)

- Placebo-in-time provides an empirical reference distribution of effects under fake dates.
- Leave-one-out highlights whether the effect is driven by a single covariate.
- Dropping last K days of pre checks local stability of the pre-fit.
- Window sensitivity shows whether conclusions depend on arbitrary window choices.
